# Single-output Use Case Demonstration
##### Author: Partha Seetala

In [1]:
import numpy as np
import pandas as pd
import random
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from keras.initializers import he_normal
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [2]:
# This function generates some synthetic data {sqft, bedrooms, lotsize and price}
def generate_data(count, noutputs=1):
    sqft_ranges = [0, 1500, 2500, 3500, 10000]
    price_ranges = [800000, 1000000, 2500000, 3000000]

    x = []
    y = []

    for i in range(count):
        sqft = random.randint(1800,4500)
        nbedrooms = 1+ int(sqft/1000)
        lotsize = int(random.uniform(1.2, 2.0) * sqft)

        price = 0
        for r in range(len(sqft_ranges)-1):
            if sqft >= sqft_ranges[r] and sqft < sqft_ranges[r+1]:
                price = price_ranges[r]
                break
        assert price != 0

        lotsize_value = int(random.randint(50000, 300000) * (lotsize/sqft))
        price = price + lotsize_value
        price = round(price, -3)

        if noutputs == 1:
            x.append([sqft, nbedrooms, lotsize])
            y.append(price)
        else:
            x.append([sqft, lotsize])
            y.append([price, nbedrooms])

    return x, y

# **Single-output Regression Use Case**


---

**Given the `sqft`, `bedrooms` and `lotsize` predict the `price` of a house**

**Let's initialize a Standardization (Normalization) module that we'll use in our model**

In [3]:
norm_x = StandardScaler() # We'll use this to normalize our input
norm_y = StandardScaler() # We'll use this to normalize our output

**Let's `build` a model to predict 1 output "`price`", given a house's "`sqft`", "`lotsize`" and "`bedrooms`"**

In [4]:
def build_model_for_one_output():
    model = Sequential()

    # Hidden Layer(s)
    model.add(Dense(10, input_dim=3, kernel_initializer=he_normal(), activation="relu"))

    # Output Layer
    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

model1 = build_model_for_one_output()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Let's generate some data**

In [5]:
# Generate some house price data
x, y = generate_data(10000, noutputs=1)

# Let's display the data we generated
data = pd.DataFrame(
    {'sqft': [rec[0] for rec in x],
    'nbedrooms': [rec[1] for rec in x],
    'lotsize': [rec[2] for rec in x],
    'price': y})
data

,sqft,nbedrooms,lotsize,price
0,2076,3,2654,1283000
1,2414,3,3847,1138000
2,3924,4,5262,3161000
3,3414,4,6341,2743000
4,2709,3,4988,2728000
...,...,...,...,...
9995,2839,3,4394,2803000
9996,4073,5,5097,3078000
9997,3870,4,4726,3068000
9998,2437,3,3866,1409000


**Let's `train` our model**

In [ ]:
def train_model_for_one_output(model, x, y):
    # Normalize input data
    xnorm = norm_x.fit_transform(np.array(x))
    ynorm = norm_y.fit_transform(np.array(y).reshape(-1,1))

    # Train the model
    model.fit(xnorm, ynorm, epochs=100, batch_size=10, verbose=1)

train_model_for_one_output(model1, x, y)

Epoch 1/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.2082
Epoch 2/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.1051
Epoch 3/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0988
Epoch 4/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0842
Epoch 5/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0763
Epoch 6/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0643
Epoch 7/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0587
Epoch 8/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0524
Epoch 9/100
 226/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0541

**Let's use our model to make predictions**

In [ ]:
def predict_model_for_one_output(model):
    x, yt = generate_data(20, noutputs=1)

    # We have to normalize the input again even during prediction because we normalized during training
    xnorm = norm_x.transform(np.array(x))

    predictions = model.predict(xnorm)

    # De-Normalize predictions
    yp = norm_y.inverse_transform(predictions).flatten()

    return x, yt, yp

x, yt, yp = predict_model_for_one_output(model1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


**Let's see how well our model did with the predictions**

In [ ]:
def show_predictions_for_one_output(x, yt, yp):
    # Pretty Print the Predictions comparing them what they should be
    offby = []
    percent = []
    for i in range(len(yp)):
        diff = int(yt[i] - yp[i])
        offby.append(diff)
        percent.append(int(100*(diff/yt[i])))

    data = pd.DataFrame(
        {'sqft': [rec[0] for rec in x],
            'nbedrooms': [rec[1] for rec in x],
            'lotsize': [rec[2] for rec in x],
            'price': yt,
            'price_pred': [int(p) for p in yp],
            'price_offby': offby,
            'price_percent_off': percent})
    return data

show_predictions_for_one_output(x, yt, yp)

,sqft,nbedrooms,lotsize,price,price_pred,price_offby,price_percent_off
0,2090,3,3567,1315000,1215230,99769,7
1,3436,4,6266,2725000,2832894,-107894,-3
2,4164,5,7191,3149000,3332420,-183420,-5
3,2569,3,4415,2891000,2736394,154605,5
4,1978,2,2532,1083000,1138648,-55648,-5
5,2180,3,3745,1235000,1210799,24200,1
6,4147,5,6770,3399000,3316247,82753,2
7,3490,4,6947,2972000,3013826,-41826,-1
8,3208,4,5027,2651000,2782349,-131349,-4
9,1943,2,2333,1280000,1130381,149619,11


**Let's initialize a Standardization module we'll use in our model**

In [ ]:
norm_x = StandardScaler() # We'll use this to standardize our input
norm_y = StandardScaler() # We'll use this to standardize our output

**Let's `build` a model to predict 2 outputs**

In [ ]:
def build_model_for_two_outputs():
    model = Sequential()

    # Hidden Layer(s)
    model.add(Dense(10, input_dim=2, kernel_initializer=he_normal(), activation="relu"))

    # Output Layer
    model.add(Dense(2))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

model2 = build_model_for_two_outputs()

**Let's generate some data**

In [ ]:

# Generate some house price data
x, y = generate_data(10000, noutputs=2)

# Let's display the data we generated
data = pd.DataFrame(
    {'X_sqft': [rec[0] for rec in x],
    'X_lotsize': [rec[1] for rec in x],
    'Y_price': [rec[0] for rec in y],
    'Y_nbedrooms': [rec[1] for rec in y]})
data

**Let's `train` our model**

In [ ]:
def train_model_for_two_outputs(model, x, y):
    # Normalize data
    xnorm = norm_x.fit_transform(np.array(x))
    ynorm = norm_y.fit_transform(np.array(y))

    # Train the model
    model.fit(xnorm, ynorm, epochs=100, batch_size=10, verbose=1)

train_model_for_two_outputs(model2, x, y)

**Let's use our model to make predictions**

In [ ]:
def predict_model_for_two_outputs(model):
    x, yt = generate_data(20, noutputs=2)

    xnorm = norm_x.transform(np.array(x))

    predictions = model.predict(xnorm)

    # De-Normalize predictions
    yp = norm_y.inverse_transform(predictions)

    return x, yt, yp

x, yt, yp = predict_model_for_two_outputs(model2)

**Let's see what our predictions look like**

In [ ]:
def show_predictions_for_two_outputs(x, yt, yp):
    price_offby = []
    price_percent = []

    nbedrooms_offby = []
    nbedrooms_percent = []

    for i in range(len(yp)):
        price_diff = int(yt[i][0] - yp[i][0])
        price_offby.append(price_diff)
        price_percent.append(int(100*(price_diff/yt[i][0])))

        nbedrooms_diff = yt[i][1] - yp[i][1]
        nbedrooms_offby.append(nbedrooms_diff)
        nbedrooms_percent.append(int(100*(nbedrooms_diff/yt[i][1])))

    data = pd.DataFrame(
            {'sqft': [rec[0] for rec in x],
            'lotsize': [rec[1] for rec in x],
            'price': [int(p[0]) for p in yt],
            'price_pred': [int(p[0]) for p in yp],
            'price_offby': price_offby,
            'price_percent_off': price_percent,
            'nbedrooms': [p[1] for p in yt],
            'nbedrooms_pred': [int(p[1]) for p in yp],
            'nbedrooms_offby': nbedrooms_offby,
            'nbedrooms_percent_of': nbedrooms_percent})
    return data

show_predictions_for_two_outputs(x, yt, yp)